# Process Inputs: Chain-of-Thought Reasoning

This notebook demonstrates **Chain-of-Thought (CoT) prompting**, a technique that significantly improves an LLM's reasoning ability by instructing it to "think step-by-step" before giving a final answer. 🧠

---
### What's Happening Here?

Instead of asking for a direct answer, we provide the model with a detailed, multi-step process in the **system prompt**. For a customer query, this process includes:
1.  Identifying the product in the query.
2.  Checking it against an internal product list.
3.  Detecting and verifying any assumptions the user makes.
4.  Formulating a polite and accurate final response.

---
### Key Takeaways

* **Improved Accuracy**: By breaking down complex problems, the model is less likely to make logical errors.
* **Interpretability**: The model outputs its entire reasoning process, separated by delimiters (`####`). This "inner monologue" allows us to see exactly how it arrived at its conclusion, which is great for debugging.
* **Final Answer Extraction**: The code demonstrates how to easily parse the model's full output to show only the clean, final response to the end-user, hiding the intermediate reasoning steps.

In [20]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [21]:
def get_completion_from_messages(messages, 
                                 model="gpt-4o", 
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens
    )
    return response.choices[0].message["content"]

## Chain-of-Thought Prompting

In [22]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [23]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking a question about specific products.

Step 2:#### The products mentioned are "BlueWave Chromebook" and "TechPro Desktop," which are both in the list of available products.

Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.

Step 4:#### Based on the product information, the BlueWave Chromebook is priced at $249.99, and the TechPro Desktop is priced at $999.99. Therefore, the assumption that the BlueWave Chromebook is more expensive than the TechPro Desktop is incorrect.

Response to user:#### It seems there was a mix-up! The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99. If you have any more questions, feel free to ask!


In [24]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is not asking about a specific product or products. They are inquiring about a product category, which is TVs.
Step 2:#### Since the user is not asking about specific products, there is no need to check against the list of available products.
Step 3:#### There are no assumptions made by the user about specific products.
Step 4:#### Since there are no assumptions, there is nothing to verify.
Response to user:#### Thank you for your inquiry! Currently, we do not sell TVs. Our store specializes in a selection of computers and laptops. If you have any questions about our available products, feel free to ask!


## Inner Monologue
- Since we asked the LLM to separate its reasoning steps by a delimiter, we can hide the chain-of-thought reasoning from the final output that the user sees.

In [25]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

Thank you for your inquiry! Currently, we do not sell TVs. Our store specializes in a selection of computers and laptops. If you have any questions about our available products, feel free to ask!


In [26]:
test_queries = [
    # 1. Incorrect assumption
    "By how much is the BlueWave Chromebook more expensive than the TechPro Desktop?",
    
    # 2. Correct assumption (about features)
    "I need the most powerful gaming machine. I assume the BlueWave Gaming Laptop is a better choice than the TechPro Desktop because of its graphics card, right?",
    
    # 3. Product not in the list
    "What is the price of a 16-inch MacBook Pro?",
    
    # 4. Simple, direct question
    "What is the warranty on the PowerLite Convertible?",
    
    # 5. Vague question
    "Tell me about the TechPro computer.",
    
    # 6. General question (not about a specific product)
    "Do you sell any computers with a touchscreen?"
]

In [28]:
test_queries = [
    "By how much is the BlueWave Chromebook more expensive than the TechPro Desktop?",
    "I need the most powerful gaming machine. I assume the BlueWave Gaming Laptop is a better choice than the TechPro Desktop because of its graphics card, right?",
    "What is the price of a 16-inch MacBook Pro?",
    "What is the warranty on the PowerLite Convertible?",
    "Tell me about the TechPro computer.",
    "Do you sell any computers with a touchscreen?"
]

# --- Processing Loop ---
for query in test_queries:
    print(f"Processing Query: '{query}'")
    print("=" * 100)
    
    user_message = query
    
    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_message}{delimiter}"},  
    ] 
    
    
    try:
        response = get_completion_from_messages(messages)
        
        print("MODEL'S FULL RESPONSE:")
        print(response)
        print("-" * 50)
        
        try:
            final_response = response.split(delimiter)[-1].strip()
        except Exception as e:
            final_response = "Sorry, I'm having trouble right now, please try asking another question."

        print("FINAL RESPONSE TO USER:")
        print(final_response)
        
    except Exception as e:
        print(f"An error occurred: {e}")
        
    print("\n" + "="*100 + "\n")

Processing Query: 'By how much is the BlueWave Chromebook more expensive than the TechPro Desktop?'
MODEL'S FULL RESPONSE:
Step 1:#### The user is asking a question about specific products.

Step 2:#### The user is asking about the BlueWave Chromebook and the TechPro Desktop, which are both in the list of available products.

Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.

Step 4:#### Based on the product information, the BlueWave Chromebook is priced at $249.99, and the TechPro Desktop is priced at $999.99. Therefore, the assumption that the BlueWave Chromebook is more expensive than the TechPro Desktop is incorrect.

Response to user:#### It seems there was a mix-up! The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook is priced at $249.99, while the TechPro Desktop is priced at $999.99. If you have any more questions or need further assistance, feel free to ask!
---------------